<h1>Using Multi Agent system for creating React app</h1>

In [ ]:
# Import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

In [1]:
openai_model="gpt-4o-mini"
deepseek_model='deepseek-chat'

In [ ]:

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from agent_instructions import * 
from agent_tools import save_app_file_structure, create_app_file_structure
from typing import Dict
import os 
import json
import asyncio
from IPython.display import display, Markdown

In [ ]:
# Innovator agent comes up with innovative ideas,to solve problems for small business.
innovator_agent = Agent(
    name="Innovator",
    instructions=innovator_instructions,
    model=openai_model
)

In [ ]:
# Idea Selector Agent selects the best idea from the list, based on solving user's problems and cost considerations.
idea_selector_agent = Agent(
    name="Idea Selector",
    instructions=idea_selector_instructions,
    model=openai_model
)

In [ ]:
app_file_structure_agent = Agent(
    name='App Structure Generator',
    instructions=app_file_structure_instructions,
    model=openai_model
)

In [ ]:
file_structure_evaluator_agent = Agent(
    name='Evaluator',
    instructions=file_structure_evaluator_instructions,
    model=openai_model,
    tool=save_app_file_structure
)

In [ ]:
component_agent = Agent(
    name='Component Generator',
    instructions=components_instructions,
    model=openai_model
)

In [ ]:
page_creator_agent = Agent(
    name='Page Generator',
    instructions=pages_instructions,
    model=openai_model
)

In [ ]:
hooks_creator_agent = Agent(
    name='Hooks Generator',
    instructions=hooks_instructions,
    model=openai_model
)

In [ ]:
database_creator_agent = Agent(
    name='Database Designer',
    instructions=database_instructions,
    model=openai_model
)

In [ ]:
api_creator_agent = Agent(
    name='API Generator',
    instructions=api_instructions,
    model=openai_model
)

In [ ]:
dependencies_generator_agent = Agent(
    name='Dependency Generator',
    instructions=dependencies_instructions,
    model=openai_model
)

In [ ]:
payment_handling_agent = Agent(
    name='Payment Handler',
    instructions=payment_instructions,
    model=openai_model
)

In [ ]:
tester_agent = Agent(
    name='QA Tester',
    instructions=tester_instructions,
    model=openai_model
)

In [ ]:
innovator_agent.handoffs = idea_selector_agent
innovator_agent.handoff_description="Please select the best idea from the list of ideas."

idea_selector_agent.handoffs= app_file_structure_agent
idea_selector_agent.handoff_description="Please create a react app files structure for the selected idea."

app_file_structure_agent.handoffs=file_structure_evaluator_agent
app-file_structure_evaluator_agent.instructions="Please evaluate the file structure for the selected idea."


In [ ]:
# Tools for developer agent
component_agent.as_tool(tool_name='Component Generator', tool_description="Use this tool to assist in writing components source code for the page directory in the file structure")
page_creator_agent.as_tool(tool_name='Page Generator', tool_description='Use this tool to assist in writing the source code for the the page directory in the file structure')
hooks_creator_agent.as_tool(tool_name='Hooks Creator', tool_description='Use this tool to assist in writing the hooks source code for the hooks directory in the file structure ')
database_creator_agent.as_tool(tool_name='Database Designer', tool_description='Use this tool to assist in designing the database schema for the app file structure')
dependencies_generator_agent.as_tool(tool_name='Dependency Generator', tool_description='Use this tool to assist in writing the dependencies for the file structure')
api_creator_agent.as_tool(tool_name='API Generator', tool_description='Use this tool to assist in writing the source code for the api section in the file structure ')
payment_handling_agent.as_tool(tool_name='Payment Handler', tool_description='Use this tool to assist in writing the source code for setting up payment in the app file structure')
tester_agent.as_tool(tool_name='QA Tester', tool_description='Use this tool to assist you testing all the source code.')


In [ ]:
# Tools for the developer agent
tools = [
    component_agent,
    page_creator_agent,
    hooks_creator_agent,
    database_creator_agent,
    dependencies_generator_agent,
    api_creator_agent,
    payment_handling_agent,
    tester_agent,
    save_app_file_structure,
    create_app_file_structure
]

In [ ]:
developer_agent = Agent(
    name='Developer', 
    instructions=developer_instructions,
    model=openai_model,
    tools=tools
)

In [ ]:
message = "Create a React app to solve the problems of small business."
with trace('Parallel Multi-Agent React App Creator'):
    results = await asyncio.gather(
        Runner.run(thinker_agent,'Come up with innovative ideas for small business.'),
        Runner.run(idea_selector,'Select the best idea from the list provided by the thinker agent.'),
        Runner.run(file_structure_agent,'Create a detailed react app file structure for the selected idea.'),
        Runner.run(developer_agent,'Write code for the login page of the app.'),
    )
    
    outputs = [result.final_output for result in results]
    
    for output in outputs:
        print(output)

Second version | 
Hierarchical Agents 

In [ ]:

developer_agent = Agent(
    name="Directory Creator",
    instructions=app_file_structure_instructions,
    model="gpt-4o-mini",
)
streamed= ""
results = Runner.run_streamed(developer_agent, developer_instruction)
async for event in results.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        streamed += event.data.delta
        #print(event.data.delta, end='', flush=True)

In [ ]:
display(Markdown(streamed))

In [ ]:
data = json.loads(streamed)
print(data)